In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

# 압축파일 경로 (Google Drive에서 불러오기)
zip_path = "/content/drive/My Drive/baby_cry_data.zip"
extract_path = "/content/drive/My Drive/baby_cry_data"

# 압축 풀기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ 압축 해제 완료! → {extract_path}")


✅ 압축 해제 완료! → /content/drive/My Drive/baby_cry_data


In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf

# 🔥 변환할 파일이 있는 폴더 (Google Drive)
input_folder = "/content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/cry"

# 🔥 변환된 파일을 저장할 폴더
output_folder = "/content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry"
os.makedirs(output_folder, exist_ok=True)  # 폴더 없으면 생성

print(f"✅ 입력 폴더: {input_folder}")
print(f"✅ 출력 폴더: {output_folder}")


✅ 입력 폴더: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/cry
✅ 출력 폴더: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry


In [ ]:
def process_audio_files(input_folder, output_folder):
    # 폴더 내 모든 .wav 파일 가져오기
    files = [f for f in os.listdir(input_folder) if f.endswith('.wav')]
    print(f"🎯 총 {len(files)}개 파일을 변환합니다.")

    chunk_count = 0  # 저장될 파일 번호

    for file in files:
        file_path = os.path.join(input_folder, file)

        # 오디오 파일 로드 (16kHz 변환)
        waveform, sr = librosa.load(file_path, sr=16000)

        # 소리 있는 부분 감지
        intervals = librosa.effects.split(waveform, top_db=30)

        for i, (start, end) in enumerate(intervals):
            segment = waveform[start:end]  # 감지된 구간만 추출

            # 1초 단위로 나누기
            chunk_size = sr * 1  # 1초 크기
            num_chunks = len(segment) // chunk_size

            for j in range(num_chunks):
                chunk = segment[j * chunk_size : (j + 1) * chunk_size]
                chunk_filename = os.path.join(output_folder, f"chunk_{chunk_count}.wav")

                # 오디오 파일 저장
                sf.write(chunk_filename, chunk, sr)
                print(f"✅ 저장 완료: {chunk_filename}")
                chunk_count += 1

# 실행
process_audio_files(input_folder, output_folder)


🎯 총 457개 파일을 변환합니다.
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_0.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_1.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_2.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_3.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_4.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_5.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset/preprocess_cry/chunk_6.wav
✅ 저장 완료: /content/drive/My Drive/baby_cry_data/baby_cry_detection_dataset/baby_cry_d

In [ ]:
import os
data_path = '/content/drive/MyDrive/baby_cry_data/baby_cry_detection_dataset/baby_cry_detection_dataset'

# 아기 울음소리와 울지 않는 소리 폴더 경로
cry_path = os.path.join(data_path, 'preprocess_cry')
not_cry_path = os.path.join(data_path, 'preprocess_not_cry')

# 각 폴더 내의 오디오 파일 경로 가져오기
cry_files = [os.path.join(cry_path, f) for f in os.listdir(cry_path) if f.endswith('.wav')]
not_cry_files = [os.path.join(not_cry_path, f) for f in os.listdir(not_cry_path) if f.endswith('.wav')]


print(f'Crying files: {len(cry_files)}')
print(f'Not crying files: {len(not_cry_files)}')


Crying files: 1194
Not crying files: 1174


In [ ]:
import tensorflow_hub as hub

# YAMNet 모델 URL
yamnet_url = 'https://tfhub.dev/google/yamnet/1'

# YAMNet 모델 불러오기
yamnet = hub.load(yamnet_url)


In [ ]:
import tensorflow as tf
import librosa
import numpy as np

def extract_features(audio_path):
    # librosa로 오디오 파일 로드 (16kHz로 리샘플링)
    audio, sr = librosa.load(audio_path, sr=16000)

    # 오디오 데이터를 텐서로 변환
    waveform = np.array(audio)  # (16000,)
    waveform = tf.convert_to_tensor(waveform, dtype=tf.float32)

    # 배치 차원을 추가하지 않고 바로 YAMNet에 전달
    # YAMNet은 (16000,) 텐서를 받아들임
    scores, embeddings, spectrogram = yamnet(waveform)  # YAMNet 모델은 (16000,) 텐서를 받음

    # embeddings를 반환 (특징 벡터)
    return embeddings.numpy()[0]  # 첫 번째 샘플의 특성 벡터 반환

# 데이터 준비
X = []
y = []

# 아기 울음소리 데이터 (cry_files)
for file in cry_files:
    feature = extract_features(file)
    X.append(feature)
    y.append(1)  # 울음소리는 레이블 1

# 울지 않는 소리 데이터 (not_cry_files)
for file in not_cry_files:
    feature = extract_features(file)
    X.append(feature)
    y.append(0)  # 울지 않는 소리는 레이블 0


In [ ]:

import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
from sklearn.model_selection import train_test_split

# X와 y는 각각 feature와 레이블을 담고 있습니다.
X = np.array(X)  # 특성 벡터들
y = np.array(y)  # 레이블들

# 훈련 데이터와 테스트 데이터로 나누기 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'훈련 데이터 크기: {len(X_train)}')
print(f'테스트 데이터 크기: {len(X_test)}')


훈련 데이터 크기: 1894
테스트 데이터 크기: 474


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# YAMNet 모델을 TensorFlow Hub에서 로드
yamnet_url = "https://tfhub.dev/google/yamnet/1"  # YAMNet 모델의 URL
yamnet = hub.load(yamnet_url)  # 모델 로드

# 모델 확인
print(yamnet.signatures['serving_default'])


ConcreteFunction Input Parameters:
  waveform (KEYWORD_ONLY): TensorSpec(shape=(None,), dtype=tf.float32, name='waveform')
Output Type:
  Dict[['output_2', TensorSpec(shape=(None, 64), dtype=tf.float32, name='output_2')], ['output_1', TensorSpec(shape=(None, 1024), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 521), dtype=tf.float32, name='output_0')]]
Captures:
  136612442519888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136612442518160: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  136612442508176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136612442515280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136612442516816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136612442520080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136610900741968: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  136612442520848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136612442521040: TensorSpec(shape=(), dty

In [ ]:
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# YAMNet의 출력층 (embeddings)을 받아서 분류기 모델을 추가합니다.
model = models.Sequential([
    layers.InputLayer(input_shape=(1024,)),

    layers.Dense(512, activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)),  # L2 추가
    layers.BatchNormalization(),
    layers.Dropout(0.6),

    layers.Dense(256, activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)),  # L2 추가
    layers.BatchNormalization(),
    layers.Dropout(0.6),

    layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 요약
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 659,457 (2.52 MB)

 Trainable params: 657,921 (2.51 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # 검증 손실 기준
    patience=2,          # 손실이 2번 연속 증가하면 중단
    min_delta=0.003,     # 최소 개선량
    restore_best_weights=True
)

# 학습률 감소 스케줄러 추가
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,   # 학습률을 절반으로 줄임
    patience=1,   # 1번 연속으로 손실이 줄어들지 않으면 학습률 감소
    min_lr=1e-6   # 최소 학습률 제한
)

# 모델 학습
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=20,  # 최대 20이나 EarlyStopping으로 자동 종료
                    batch_size=32,
                    callbacks=[early_stopping, lr_scheduler],  # 두 개 다 적용
                    verbose=1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.8570 - loss: 7.9152 - val_accuracy: 0.9705 - val_loss: 3.4608 - learning_rate: 0.0010
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.9780 - loss: 2.9873 - val_accuracy: 0.9705 - val_loss: 2.0723 - learning_rate: 0.0010
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.9812 - loss: 1.8278 - val_accuracy: 0.9831 - val_loss: 1.3931 - learning_rate: 0.0010
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/step - accuracy: 0.9835 - loss: 1.2707 - val_accuracy: 0.9789 - val_loss: 1.0257 - learning_rate: 0.0010
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.9802 - loss: 0.9474 - val_accuracy: 0.9810 - val_loss: 0.7467 - learning_rate: 0.0010
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 111ms/step - accuracy: 0.9835 - loss: 0.7130 - val_accuracy: 0.9810 - val_loss: 0.6187 - learning_rate: 0.0010
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.9792 - loss: 0.5876 - val_accura

In [ ]:
# 테스트 데이터에서 모델 성능 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'테스트 손실: {test_loss}')
print(f'테스트 정확도: {test_acc}')

15/15 - 0s - 21ms/step - accuracy: 0.9810 - loss: 0.2791
테스트 손실: 0.2790680229663849
테스트 정확도: 0.9810126423835754


In [ ]:
!pip install tensorflow tensorflow_io


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 22.7 MB/s eta 0:00:00


In [ ]:
model.save('my_model.h5')  # HDF5 형식으로 저장


In [ ]:
import tensorflow as tf

# 저장된 모델 로드
model = tf.keras.models.load_model("/content/my_model.h5")
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
import numpy as np

# 1️⃣ YAMNet 모델 로드
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

# 2️⃣ 저장된 울음소리 분류 모델 로드
model = tf.keras.models.load_model("/content/my_model.h5")

def preprocess_audio(file_path):
    file_data = tf.io.read_file(file_path)
    audio, sample_rate = tf.audio.decode_wav(file_data, desired_channels=1)
    audio = tf.squeeze(audio, axis=-1)  # (N, 1) -> (N,)
    return audio  # ⚠️ (1, N) → (N,) 형태 유지

# 4️⃣ 예측 함수
def predict_cry(file_path):
    # 오디오 전처리
    waveform = preprocess_audio(file_path)

    # YAMNet을 사용하여 embedding 추출
    _, embeddings, _ = yamnet_model(waveform)  # ⚠️ 오류 해결: (None,) 형태 유지

    # 우리의 모델에 입력
    prediction = model.predict(embeddings)

    # 평균값을 이용하여 최종 판별
    cry_probability = np.mean(prediction)

    # 0.5 이상이면 울음으로 판별
    if cry_probability > 0.5:
        print(f"울음 감지! (확률: {cry_probability:.2f})")
    else:
        print(f"울음 없음 (확률: {cry_probability:.2f})")


# 5️⃣ 예제 실행
predict_cry("/content/test.wav")  # 테스트할 음성 파일 경로


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
울음 없음 (확률: 0.00)
